## ■ 강화학습을 이용한 tic tac toe 코드 구현

"강화"는 동물이 시행착오를 통해 학습하는 방법중 하나로 강화라는 개념을 처음 제시한 것은 스키너라는 행동심리학자이다.  
"강화"라는 것은 동물이 이전에 배우지 않았지만 직접 시도하면서 행동과 그 결과로 나타나는 좋은 보상 사이에 상관관계를 학습하는 것이다.  

구글에서 스키너의 쥐실험!

"강화"의 핵심은 바로 보상을 얻게 해주는 __행동의 빈도의 증가__입니다.  
다른 말로 얘기하면 보상을 얻게하는 행동을 점점 더 많이 하도록 학습하는것을 말한다.  

## ■ 지도학습과 강화학습의 차이는?

지도학습은 직접적인 정답을 통해 오차를 계산해서 학습했지만,  
강화학습은 자신의 행동의 결과로 나타나는 보상을 통해 학습합니다.  

## ■ 학습에 필요한 요소는??

### "MDP(MarKov Dicision Process)"

1. 상태
2. 행동
3. 보상
4. 상태변이확률
5. 감가율  

## ■ 틱텍토 환경 구성  

#### 1. 틱텍토 기본게임이 만들어지기 위한 함수
    - printborad : 보드판을 출력하는 함수
    - emptystate : 보드판을 초기화하는 함수
    - gameover : 이겼는지 졌는지 비겼는지 출력하는 함수
    - action : 게임의 수를 두는 함수
    - play : 게임 순서대로 action을 수행하겠금 하는 함수
    - episode_over : 게임결과를 출력하는 함수
    
    
#### 2. 강화학습이 이뤄지기 위해 필요한 함수  
    - statetuple : 학습 데이터를 생성하는 함수
    - add : 학습 데이터를 딕셔너리변수에 추가하는 함수
    - lookup : 지금 상황에서 어느 수가 최선인지를 학습 데이터에서 찾는 함수
    - winnerval : 보상해주는 함수(1, -1, 0.5)
    - greedy : 남아있는 수들 중에서 어떤수가 가장 좋은수인지를 알려주는 함수
    - backup : 학습 데이터의 보상숫자를 특정 수학공식으로 갱신해주는 함수  
    "가중치 = 가중치 + (0.99 * 가중치의 변화량)" <-- 0.99(감가율)

## ■ 3000번 스스로 학습하는 인공지능 틱텍토 

In [ ]:

import random, csv
from copy import copy, deepcopy
# deepcopy : 메모리를 완전히 새롭게 생성
# copy : 껍데기만 카피, 내용은 동일한 곳을 가리킴

# Part1. 보드판을 그리는 부분

EMPTY = 0
PLAYER_X = 1
PLAYER_O = 2
DRAW = 3
BOARD_FORMAT = "----------------------------\n| {0} | {1} | {2} |\n|--------------------------|\n| {3} | {4} | {5} |\n|--------------------------|\n| {6} | {7} | {8} |\n----------------------------"
NAMES = [' ', 'X', 'O']


# part2. 보드판을 완성하는 코드
# 보드 출력


def printboard(state):
    cells = []
    for i in range(3):
        for j in range(3):
            cells.append(NAMES[state[i][j]].center(6))
            # center(6) 판의 가운데에 예쁘게 들어가기 위해 넣음.
            # NAMES = [' ', 'X', 'O']
            
    # print ( cells) # ['  X   ', '  O   ', '      ', '      ', '      ', '      ', '      ', '      ', '      ']       
    # print (*cells) #   X      O                                                    
    print(BOARD_FORMAT.format(*cells))



# part3. 비어있는 판을 출력하는 empty() 함수
# 새로운 판을 만들어서 reset할 때 쓰임 
# 빈 판

def emptystate():
    return [[EMPTY, EMPTY, EMPTY], [EMPTY, EMPTY, EMPTY], [EMPTY, EMPTY, EMPTY]]

# print(emptystate())
# [[0, 0, 0], [0, 0, 0], [0, 0, 0]]



# Part4. 게임에서 누가 이겼는지를 확인하는 gameover() 함수 
def gameover(state):

    # 가로/세로로 한 줄 완성한 플레이어가 있다면 그 플레이어 리턴

    for i in range(3):

        if state[i][0] != EMPTY and state[i][0] == state[i][1] and state[i][0] == state[i][2]:

            return state[i][0]

        if state[0][i] != EMPTY and state[0][i] == state[1][i] and state[0][i] == state[2][i]:

            return state[0][i]

    # 좌우 대각선

    if state[0][0] != EMPTY and state[0][0] == state[1][1] and state[0][0] == state[2][2]:

        return state[0][0]

    if state[0][2] != EMPTY and state[0][2] == state[1][1] and state[0][2] == state[2][0]:

        return state[0][2]

    # 판이 비었는지

    for i in range(3):

        for j in range(3):

            if state[i][j] == EMPTY:

                return EMPTY

    return DRAW

# 사람

class Human(object):

    def __init__(self, player):

        self.player = player



    # Part5. 게임 순서대로 action을 수행하겠끔 하는 play(함수)
    # play 함수 안에 있는 action 함수 
    # " action : 게임의 수를 두는 함수 " 

    def action(self, state):

        printboard(state)

        action = None

        while action not in range(1, 10):

            action = int(input('Your move? '))

        switch_map = {

            1: (0, 0),

            2: (0, 1),

            3: (0, 2),

            4: (1, 0),

            5: (1, 1),

            6: (1, 2),

            7: (2, 0),

            8: (2, 1),

            9: (2, 2)

        }

        return switch_map[action]




    def episode_over(self, winner):

        if winner == DRAW:

            print('Game over! It was a draw.')

        else:

            print('Game over! Winner: Player {0}'.format(winner))







def play(agent1, agent2):

    state = emptystate()

    for i in range(9):

        if i % 2 == 0:

            move = agent1.action(state)

        else:

            move = agent2.action(state)

        state[move[0]][move[1]] = (i % 2) + 1

        winner = gameover(state)
        print(winner)

        if winner != EMPTY:

            return winner

    return winner




class Computer(object):

    def __init__(self, player):

        self.player = player

        self.values = {} # csv 에 있는 파일의 내용(9개의 판(수)과 가중치)를 읽어서 저장할 딕셔너리 변수

        self.readCSV() # init 할때 values 에 값 채워넣을려고 함수를 실행함

        self.verbose = True

        #print(self.values) # {((0, 2, 1), (2, 2, 0), (1, 1, 0)): -0.999999,...

                           #  위와 같이 values 딕셔너리 변수에 저장되어 있는지 확인한다.




    def readCSV(self):

        file = open("D:\\data\\ttt_learn_data.csv", 'r')

        ttt_list = csv.reader(file)

        for t in ttt_list:

            try:

                self.values[((int(t[0]) ,int(t[1]) ,int(t[2])),(int(t[3]) ,int(t[4]) ,int(t[5])) ,(int(t[6]) ,int(t[7])

                             ,int(t[8])))] = float(t[10])

            except ValueError:    # {((0, 2, 1), (2, 2, 0), (1, 1, 0)): -0.999999,..

                continue



    def random(self, state):  # 남아있는 비어 있는 수들 중에서 한수를 random 으로 고르기 위한 함수

        available = []

        for i in range(3):

            for j in range(3):

                if state[i][j] == EMPTY:

                    available.append((i ,j))

        return random.choice(available)




    def greedy(self, state):

        maxval = -50000  # 남아있는 수중에 가장 좋은 수의 가중치를 담기 위해  선언

        maxmove = None   # 남아있는 수중에 가장 좋은 수를 담기 위해 선언

        if self.verbose:  # 수를 둘때마다 남아있는 수들의 확률을 확인하기 위해서 사용하는 코드

            cells = []

        for i in range(3):

            for j in range(3):

                if state[i][j] == EMPTY: # 남아있는 수중에 비어있는 수를 찾아서

                    state[i][j] = self.player # 거기에 플레이어의 숫자를 넣은후

                    val = self.lookup(state) # values 에 없으면 새로 0.5 를

                    #print(val)               # values 에 넣어주고 그 값을 다시 여기로 가져온다

                                              # 있으면 바로 values 에서 가져온다. (-0.9606 )

                    state[i][j] = EMPTY      # 그 수를 다시 비워준다.




                    if val > maxval:

                        maxval = val

                        # print (maxval) # 남아있는 수중에 가장 큰게 0.029698 (0.030) 이었음

                        maxmove = (i, j)

                        #print(maxmove)   # 남아있는 수중에 가장 가장치가 큰 자리 (2,0)

                    if self.verbose:  #

                        cells.append('{0:.3f}'.format(val).center(6))

                elif self.verbose:

                    cells.append(NAMES[state[i][j]].center(6))

        if self.verbose:

            print (BOARD_FORMAT.format(*cells))

           # ---------------------------    verbose 는 이 결과를 출력하기 위한 코드임

           # | 0.000 | -1.000 | 0.000 |

           # | -------------------------- |

           # | -1.000 | X | -0.961 |

           # | -------------------------- |

           # | 0.030 | -1.000 | 0.000 |

           # ----------------------------

        # print(maxmove)  # (2,0) 을 출력 ( 남아있는 수중에 가장 좋은수 )

        return maxmove




    def lookup(self, state):

        key = self.statetuple(state) # 리스트를 튜플로 바꿔주는 역활

        #print(key)  # x (player 1) 가 5번에 두었을때 o (player 2) 가 둘수있는 남아있는 수 출력

        # ((2, 0, 0), (0, 1, 0), (0, 0, 0))

        # ((0, 2, 0), (0, 1, 0), (0, 0, 0))

        # ((0, 0, 2), (0, 1, 0), (0, 0, 0))

        # ((0, 0, 0), (2, 1, 0), (0, 0, 0))

        # ((0, 0, 0), (0, 1, 2), (0, 0, 0))

        # ((0, 0, 0), (0, 1, 0), (2, 0, 0))

        # ((0, 0, 0), (0, 1, 0), (0, 2, 0))

        # ((0, 0, 0), (0, 1, 0), (0, 0, 2))

        if not key in self.values: # 위의  key 수들이 csv 에서 읽어온 수들중에 없다면

            self.add(key)  # values 에 없으며 add 함수로 추가

        #print (self.values) # {((0, 2, 1), (2, 2, 0), (1, 1, 0)): -0.999999, ...

        #print (self.values[key]) # -0.999847, 0.0, -0.999996, .......

        return self.values[key]  # 있으면 그거 리턴, 없으면 만들고 리턴




    def add(self, state):

        winner = gameover(state)

        tup = self.statetuple(state)

        self.values[tup] = self.winnerval(winner) # 1,-1,0.5, 0 (비긴것)




    def statetuple(self, state):

        return (tuple(state[0]) ,tuple(state[1]) ,tuple(state[2]))




    # 컴퓨터가 착수

    def action(self, state):

        printboard(state)

        action = None

        move = self.greedy(state)

        state[move[0]][move[1]] = self.player

        return move




    def winnerval(self, winner):

        if winner == self.player:

            return 1

        elif winner == EMPTY:

            return 0.5

        elif winner == DRAW:

            return 0

        else:

            return self.lossval




    def episode_over(self, winner):

        if winner == DRAW:

            print('Game over! It was a draw.')

        else:

            print('Game over! Winner: Player {0}'.format(winner))




if __name__ == "__main__":

    p1 = Human(1)

    p2 = Computer(2)

    while True:

        winner = play(p1, p2)

        p1.episode_over(winner)

        p2.episode_over(winner)

### 틱텍토의 경우 칸이 9칸 밖에 없기 때문에 9! = 362880 개의 경우의 수가 있다.  
### 컴퓨터가 학습을 하게 되면 사람하고 대결했을 때 계속 비긴다. 

## ■ 틱텍토 인공지능 바로 전 코드

링크 : http://cafe.daum.net/oracleoracle/SZTZ/1964

강화학습 코드설명_틱택토 전체.pdf 참고

In [ ]:
##■ 틱텍토 사람대 인공지능_20190823.py (이론 설명 참고 코드)

import random, csv

from copy import copy, deepcopy

# deepcopy : 메모리를 완전히 새롭게 생성
# copy : 껍데기만 카피, 내용은 동일한 곳을 가리킴


# Part1. 보드판을 그리는 부분

EMPTY = 0
PLAYER_X = 1
PLAYER_O = 2
DRAW = 3

BOARD_FORMAT = "----------------------------\n| {0} | {1} | {2} |\n|--------------------------|\n| {3} | {4} | {5} |\n|--------------------------|\n| {6} | {7} | {8} |\n----------------------------"

NAMES = [' ', 'X', 'O']

print(BOARD_FORMAT)

# part2. 보드판을 완성하는 코드
# 보드 출력

def printboard(state):
    cells = []
    for i in range(3):
        for j in range(3):
            cells.append(NAMES[state[i][j]].center(6))
            # center(6) 판의 가운데에 예쁘게 들어가기 위해 넣음.
            # NAMES = [' ', 'X', 'O']
            
    # print(cells) # ['  X   ', '  O   ', '      ', '      ', '      ', '      ', '      ', '      ', '      ']
    # print(*cells) # X      O  : 리스트에 있는 요소들을 출력해준다.
    print(BOARD_FORMAT.format(*cells))

state = [[1,2,0],[0,0,0],[0,0,0]]
print(printboard(state))


# part3. 비어있는 판을 출력하는 empty() 함수
# 새로운 판을 만들어서 reset할 때 쓰임 
# 빈 판

def emptystate():

    return [[EMPTY, EMPTY, EMPTY], [EMPTY, EMPTY, EMPTY], [EMPTY, EMPTY, EMPTY]]

# print(emptystate()) # [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

state = emptystate()
def printboard(state):
    cells = []
    for i in range(3):
        for j in range(3):
            cells.append(NAMES[state[i][j]].center(6))
            # center(6) 판의 가운데에 예쁘게 들어가기 위해 넣음.
            # NAMES = [' ', 'X', 'O']
    print(BOARD_FORMAT.format(*cells))
    
print(printboard(state))

def gameover(state):

    # 가로/세로로 한 줄 완성한 플레이어가 있다면 그 플레이어 리턴

    for i in range(3):

        if state[i][0] != EMPTY and state[i][0] == state[i][1] and state[i][0] == state[i][2]:

            return state[i][0]

        if state[0][i] != EMPTY and state[0][i] == state[1][i] and state[0][i] == state[2][i]:

            return state[0][i]
        
    print(state)
    # 좌우 대각선

    if state[0][0] != EMPTY and state[0][0] == state[1][1] and state[0][0] == state[2][2]:

        return state[0][0]

    if state[0][2] != EMPTY and state[0][2] == state[1][1] and state[0][2] == state[2][0]:

        return state[0][2]
    
    # 판이 비었는지

    for i in range(3):

        for j in range(3):

            if state[i][j] == EMPTY:

                return EMPTY

    return DRAW


# 사람

class Human(object):

    def __init__(self, player):

        self.player = player


    # 착수

    def action(self, state):

        printboard(state)

        action = None

        while action not in range(1, 10):

            action = int(input('Your move? '))

        switch_map = {

            1: (0, 0),

            2: (0, 1),

            3: (0, 2),

            4: (1, 0),

            5: (1, 1),

            6: (1, 2),

            7: (2, 0),

            8: (2, 1),

            9: (2, 2)

        }

        return switch_map[action]




    def episode_over(self, winner):

        if winner == DRAW:

            print('Game over! It was a draw.')

        else:

            print('Game over! Winner: Player {0}'.format(winner))







def play(agent1, agent2):

    state = emptystate()

    for i in range(9):

        if i % 2 == 0:

            move = agent1.action(state)

        else:

            move = agent2.action(state)

        state[move[0]][move[1]] = (i % 2) + 1

        winner = gameover(state)

        if winner != EMPTY:

            return winner

    return winner




class Computer(object):

    def __init__(self, player):

        self.player = player

        self.values = {} # csv 에 있는 파일의 내용(9개의 판(수)과 가중치)를 읽어서 저장할 딕셔너리 변수

        self.readCSV() # init 할때 values 에 값 채워넣을려고 함수를 실행함

        self.verbose = True

        #print(self.values) # {((0, 2, 1), (2, 2, 0), (1, 1, 0)): -0.999999,...

                           #  위와 같이 values 딕셔너리 변수에 저장되어 있는지 확인한다.




    def readCSV(self):

        file = open("D:\\data\\ttt_learn_data.csv", 'r')

        ttt_list = csv.reader(file)

        for t in ttt_list:

            try:

                self.values[((int(t[0]) ,int(t[1]) ,int(t[2])),(int(t[3]) ,int(t[4]) ,int(t[5])) ,(int(t[6]) ,int(t[7])

                             ,int(t[8])))] = float(t[10])

            except ValueError:    # {((0, 2, 1), (2, 2, 0), (1, 1, 0)): -0.999999,..

                continue




    def random(self, state):  # 남아있는 비어 있는 수들 중에서 한수를 random 으로 고르기 위한 함수

        available = []

        for i in range(3):

            for j in range(3):

                if state[i][j] == EMPTY:

                    available.append((i ,j))

        return random.choice(available)




    def greedy(self, state):

        maxval = -50000  # 남아있는 수중에 가장 좋은 수의 가중치를 담기 위해  선언

        maxmove = None   # 남아있는 수중에 가장 좋은 수를 담기 위해 선언

        if self.verbose:  # 수를 둘때마다 남아있는 수들의 확률을 확인하기 위해서 사용하는 코드

            cells = []

        for i in range(3):

            for j in range(3):

                if state[i][j] == EMPTY: # 남아있는 수중에 비어있는 수를 찾아서

                    state[i][j] = self.player # 거기에 플레이어의 숫자를 넣은후

                    val = self.lookup(state) # values 에 없으면 새로 0.5 를

                    #print(val)               # values 에 넣어주고 그 값을 다시 여기로 가져온다

                                              # 있으면 바로 values 에서 가져온다. (-0.9606 )

                    state[i][j] = EMPTY      # 그 수를 다시 비워준다.




                    if val > maxval:

                        maxval = val

                        # print (maxval) # 남아있는 수중에 가장 큰게 0.029698 (0.030) 이었음

                        maxmove = (i, j)

                        #print(maxmove)   # 남아있는 수중에 가장 가장치가 큰 자리 (2,0)

                    if self.verbose:  #

                        cells.append('{0:.3f}'.format(val).center(6))

                elif self.verbose:

                    cells.append(NAMES[state[i][j]].center(6))

        if self.verbose:

            print (BOARD_FORMAT.format(*cells))

           # ---------------------------    verbose 는 이 결과를 출력하기 위한 코드임

           # | 0.000 | -1.000 | 0.000 |

           # | -------------------------- |

           # | -1.000 | X | -0.961 |

           # | -------------------------- |

           # | 0.030 | -1.000 | 0.000 |

           # ----------------------------

        # print(maxmove)  # (2,0) 을 출력 ( 남아있는 수중에 가장 좋은수 )

        return maxmove




    def lookup(self, state):

        key = self.statetuple(state) # 리스트를 튜플로 바꿔주는 역활

        #print(key)  # x (player 1) 가 5번에 두었을때 o (player 2) 가 둘수있는 남아있는 수 출력

        # ((2, 0, 0), (0, 1, 0), (0, 0, 0))

        # ((0, 2, 0), (0, 1, 0), (0, 0, 0))

        # ((0, 0, 2), (0, 1, 0), (0, 0, 0))

        # ((0, 0, 0), (2, 1, 0), (0, 0, 0))

        # ((0, 0, 0), (0, 1, 2), (0, 0, 0))

        # ((0, 0, 0), (0, 1, 0), (2, 0, 0))

        # ((0, 0, 0), (0, 1, 0), (0, 2, 0))

        # ((0, 0, 0), (0, 1, 0), (0, 0, 2))

        if not key in self.values: # 위의  key 수들이 csv 에서 읽어온 수들중에 없다면

            self.add(key)  # values 에 없으며 add 함수로 추가

        #print (self.values) # {((0, 2, 1), (2, 2, 0), (1, 1, 0)): -0.999999, ...

        #print (self.values[key]) # -0.999847, 0.0, -0.999996, .......

        return self.values[key]  # 있으면 그거 리턴, 없으면 만들고 리턴




    def add(self, state):

        winner = gameover(state)

        tup = self.statetuple(state)

        self.values[tup] = self.winnerval(winner) # 1,-1,0.5, 0 (비긴것)




    def statetuple(self, state):

        return (tuple(state[0]) ,tuple(state[1]) ,tuple(state[2]))




    # 컴퓨터가 착수

    def action(self, state):

        printboard(state)

        action = None

        move = self.greedy(state)

        state[move[0]][move[1]] = self.player

        return move




    def winnerval(self, winner):

        if winner == self.player:

            return 1

        elif winner == EMPTY:

            return 0.5

        elif winner == DRAW:

            return 0

        else:

            return self.lossval




    def episode_over(self, winner):

        if winner == DRAW:

            print('Game over! It was a draw.')

        else:

            print('Game over! Winner: Player {0}'.format(winner))




if __name__ == "__main__":

    p1 = Human(1)

    p2 = Human(2)

    while True:

        winner = play(p1, p2)

        p1.episode_over(winner)

        p2.episode_over(winner)